In [104]:
import pandas as pd
import numpy as np
from gurobipy import *

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from PIL import Image

import csv


In [105]:
path = './testdata/'
date_version = '0905'
instance_id = '4'
min_version = '_15'
gurobi_runtime = 60*20 

Daily_Order = pd.read_csv(path + 'testdata' + instance_id + '.csv', index_col='ID')
Daily_Order.head()
L = pd.read_csv(path + 'L_jk' + min_version + '.csv', index_col='SET')
P = pd.read_csv(path + 'P_k.csv', index_col='exam')
Open_Period = pd.read_csv(path + 'Open_Period' + min_version + '.csv', index_col='exam')
End_Period = pd.read_csv(path + 'End_Period' + min_version + '.csv', index_col='exam')
On_Duty = pd.read_csv(path + 'On_Duty' + min_version +
                      '.csv', index_col='doctor id')


In [106]:
# n
n_I = Daily_Order.shape[0]
n_J = 6
n_K = 7
n_T = (24-8)*60//15
# t 從 8:00 計算到 24:00


In [107]:
# Sets
I = range(1, n_I+1)
J = range(1, n_J+1)
K = range(1, n_K+1)
T = range(1, n_T+1)
T2 = range(0, n_T+9)
Exam = {1: 'Register', 2: 'MR', 3: 'US', 4: 'CT', 5: 'Meal', 6: 'Interpretation', 7: 'Consultation'}


In [108]:
# Parameters

# 顧客 i 選擇的套餐
Choose_ij = Daily_Order['SET'] # [id]: 套餐

S_ij = {}  # Boolean
for i in I:
    for j in J:
        if Choose_ij[i] == j:
            S_ij[i, j] = 1
        else:
            S_ij[i, j] = 0


M_i = Daily_Order['MEAL_YN'] # Boolean
C_i = Daily_Order['CT_YN']  # Boolean

# Set K_i is made with M_i and C_i
K_i = {} # 
for i in I:
    K_i[i] = [1,2,3,4,5,6,7]
    if not C_i[i]:
        K_i[i].remove(4)
    if not M_i[i]:
        K_i[i].remove(5)

# # 檢查項目和套餐的配對
# B_jk = np.ones((n_J, n_K))
# B_jk[:, 3:5] = 0
# B_jk = pd.DataFrame(B_jk, index=J, columns=K)

# 各套餐所需 period 數
L_jk = L

# 用餐、CT、Interpretation 及 buffer 的時長
T_M = L.loc[1]['Meal']
T_C = L.loc[1]['CT']
T_I = L.loc[1]['Interpretation']
T_B = 1

# 各檢查項目一個時段可以服務多少客人
P_k = P
P_k.index = [1,2,3,4,5]

# 各套餐所需的時長
T_j = L_jk.sum(axis=1) - T_M - T_C

# 各檢查項目開放的時間及希望結束的時間
O_S_k = Open_Period
O_E_k = End_Period

# 各項目超時的懲罰權重
W_k = {}
for k in K:
    W_k[Exam[k]] = 3
W_k = pd.DataFrame.from_dict(W_k, orient='index', columns=['Value'])

# 每個顧客做每個檢查項目所需的時長(已將不做 CT 或用餐的時間考慮進去)
# D_ik 是一個 df
D_ik = {}
for i in I:
    D_ik[i] = L_jk.loc[Choose_ij[i]]
D_ik = pd.DataFrame.from_dict(D_ik, orient='index')
for i in I:
    if not C_i[i]:
        D_ik.loc[i]['CT'] = 0
    if not M_i[i]:
        D_ik.loc[i]['Meal'] = 0

# 各顧客真正做檢查的時長
R_i = {}
for i in I:
    R_i = D_ik.sum(axis=1)-D_ik['Interpretation']


In [109]:
On_Duty
n_DR = len(On_Duty)
DR = range(1,n_DR+1)
On_Duty['start period'][1]


1

In [110]:
Doctor_On_Duty = {}
for t in T:
    Doctor_On_Duty[t] = 0

for dr in DR:
    for t in range(On_Duty['start period'][dr], On_Duty['end period'][dr]+1):
        Doctor_On_Duty[t] += 1


In [111]:
Doctor_On_Duty

{1: 3,
 2: 3,
 3: 3,
 4: 3,
 5: 3,
 6: 3,
 7: 3,
 8: 3,
 9: 3,
 10: 3,
 11: 3,
 12: 3,
 13: 3,
 14: 3,
 15: 3,
 16: 3,
 17: 3,
 18: 3,
 19: 3,
 20: 3,
 21: 3,
 22: 3,
 23: 3,
 24: 3,
 25: 3,
 26: 3,
 27: 3,
 28: 3,
 29: 3,
 30: 3,
 31: 3,
 32: 3,
 33: 3,
 34: 3,
 35: 3,
 36: 3,
 37: 3,
 38: 3,
 39: 3,
 40: 3,
 41: 3,
 42: 1,
 43: 1,
 44: 1,
 45: 1,
 46: 1,
 47: 1,
 48: 1,
 49: 1,
 50: 1,
 51: 1,
 52: 1,
 53: 1,
 54: 1,
 55: 1,
 56: 1,
 57: 1,
 58: 1,
 59: 1,
 60: 1,
 61: 1,
 62: 1,
 63: 1,
 64: 1}

In [112]:
# Gurobi var
model = Model("model_" + date_version)
x_ikt = model.addVars(I, K, T, vtype=GRB.BINARY, name="x")
y_ikt = model.addVars(I, K, T, vtype=GRB.BINARY, name="y")
v_ik = model.addVars(I, K, vtype=GRB.BINARY, name="v")
w_ikt = model.addVars(I, K, T2, vtype=GRB.BINARY, name="w") # 項目結束時間
z_ik = model.addVars(I, K, vtype=GRB.INTEGER, name="z")
s_i = model.addVars(I, vtype=GRB.INTEGER, name="s")
e_i = model.addVars(I, vtype=GRB.INTEGER, name="e")
u_ik = model.addVars(I,K, vtype=GRB.INTEGER, name="u")
model.update()


In [113]:
# Objective function
model.setObjective(quicksum(e_i[i]-s_i[i]-R_i[i]+quicksum(u_ik[i,k]*W_k.loc[Exam[k]] for k in K_i[i]) for i in I))


In [114]:
# Constraints
# 顧客檢查項目的順序
model.addConstrs(z_ik[i,1] <= z_ik[i,k] for i in I for k in K_i[i])
model.addConstrs(quicksum(y_ikt[i,4,t] for t in T) == C_i[i] for i in I)
model.addConstrs(quicksum(y_ikt[i,5,t] for t in T) == M_i[i] for i in I)

model.addConstrs(quicksum(y_ikt[i,2,s] for s in range(1,t)) >= y_ikt[i,7,t] for i in I for t in T)
model.addConstrs(quicksum(y_ikt[i,3,s] for s in range(1,t)) >= y_ikt[i,7,t] for i in I for t in T)
model.addConstrs(quicksum(y_ikt[i,3,s] for s in range(1,t)) >= M_i[i] * y_ikt[i,5,t] for i in I for t in T)
model.addConstrs(quicksum(y_ikt[i, 4, s] for s in range(1, t)) >= M_i[i] * C_i[i] * y_ikt[i,5,t] for i in I for t in T)
model.addConstrs(quicksum(y_ikt[i,2,s] for s in range(1,t)) >= y_ikt[i,6,t] for i in I for t in T)
model.addConstrs(quicksum(y_ikt[i,3,s] for s in range(1,t)) >= y_ikt[i,6,t] for i in I for t in T)
model.addConstrs(quicksum(y_ikt[i,6,s] for s in range(1,t)) >= y_ikt[i,7,t] for i in I for t in T)

# 新增項目（處理 6 於 2,3 結束前開始）
model.addConstrs(z_ik[i,2] + D_ik['MR'][i] <= z_ik[i,6] for i in I)
model.addConstrs(z_ik[i,3] + D_ik['US'][i] <= z_ik[i,6] for i in I)
model.addConstrs(z_ik[i,6] + D_ik['Interpretation'][i] <= z_ik[i,7] for i in I)




{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>}

In [115]:
P_k

,period
1,2
2,2
3,2
4,1
5,999


In [116]:
# Constraints
# 一個人一個時間最多只能做一件事
model.addConstrs(quicksum(x_ikt[i,k,t] for k in (set(K_i[i])-{6})) <= 1 for i in I for t in T)
# 一個項目一個時間不能超過上限
model.addConstrs(quicksum(x_ikt[i,k,t] for i in I) <= P_k.loc[k] for k in range(1,6) for t in T)

# 要改
# 醫生一個時間不能超過上限
# model.addConstrs(quicksum(x_ikt[i,6,t] + x_ikt[i,7,t] for i in I) <= P_k.loc[6] for t in T)
# 醫生彈性選擇上班時段
model.addConstrs(quicksum(x_ikt[i,6,t] + x_ikt[i,7,t] for i in I) <= Doctor_On_Duty[t] for t in T)



# y x 連接
model.addConstrs(quicksum(x_ikt[i, k, s] for s in range(t, t+D_ik.loc[i][Exam[k]])) >=
                 D_ik.loc[i][Exam[k]]*y_ikt[i, k, t] for i in I for k in K_i[i] for t in range(1,n_T-D_ik.loc[i][Exam[k]]))

model.addConstrs(quicksum(y_ikt[i,k,t] for t in T)==1 for i in I for k in K_i[i])
model.addConstrs(z_ik[i,k] >= O_S_k.loc[Exam[k]] for i in I for k in K_i[i])
model.addConstrs(z_ik[i,k] == quicksum(y_ikt[i,k,t]*t for t in T) for i in I for k in K_i[i])
model.addConstrs(s_i[i] == z_ik[i,1] for i in I)
model.addConstrs(e_i[i] >= z_ik[i,k]+D_ik.loc[i][Exam[k]] for i in I for k in K_i[i])
model.addConstrs(u_ik[i,k] >= 0 for i in I for k in K_i[i])
model.addConstrs(u_ik[i,k] >= z_ik[i,k]+D_ik.loc[i][Exam[k]]-O_E_k.loc[Exam[k]] for i in I for k in K_i[i])

# 新增項目（buffer with w）

model.addConstrs(y_ikt[i,k,t] == w_ikt[i,k,t+D_ik.loc[i][Exam[k]]-1]  for i in I for k in K for t in (set(T)-{64}))
model.addConstrs(quicksum(w_ikt[i,k,t] + x_ikt[i,k,t+1] for i in I) <= P_k.loc[k] for k in (set(K_i[i])-{5,6,7}) for t in (set(T)-{64}))
model.addConstrs(quicksum(w_ikt[i,k,t] + x_ikt[i,k,t+1] for i in I for k in {6,7} ) <= Doctor_On_Duty[t] for t in (set(T)-{64}))

# 新增項目（測試）
model.addConstrs(e_i[i]-s_i[i]-R_i[i] >= 0 for i in  I)



{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>}

In [117]:
model.update()
model.params.TimeLimit = gurobi_runtime
model.optimize()


Set parameter TimeLimit to value 1200
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 14260 rows, 14300 columns and 191210 nonzeros
Model fingerprint: 0x21ef9788
Variable types: 0 continuous, 14300 integer (14140 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [1e+00, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 7010 rows and 7514 columns
Presolve time: 0.16s
Presolved: 7250 rows, 6786 columns, 115031 nonzeros
Variable types: 0 continuous, 6786 integer (6656 binary)

Root relaxation: objective 2.842171e-14, 1364 iterations, 0.04 seconds (0.07 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0  277          -    0.00000      -     - 

In [118]:
for i in range(1,2):
    for k in range(1,8):
        for t in T:
            if x_ikt[i, k, t].x >0:
                print(i,k,t)
    # print(i,R_i[i])

1 1 15
1 1 16
1 1 56
1 1 57
1 1 58
1 1 59
1 1 60
1 1 61
1 1 62
1 2 20
1 2 21
1 3 17
1 4 18
1 4 19
1 5 2
1 5 3
1 5 4
1 5 5
1 5 6
1 5 7
1 5 8
1 5 9
1 5 10
1 5 11
1 5 12
1 5 13
1 5 14
1 5 22
1 5 23
1 5 27
1 5 28
1 5 29
1 5 30
1 5 31
1 5 32
1 5 33
1 5 34
1 5 35
1 5 36
1 5 37
1 5 38
1 5 39
1 5 40
1 5 41
1 5 42
1 5 43
1 5 44
1 5 45
1 5 46
1 5 47
1 5 48
1 5 49
1 5 50
1 5 51
1 5 52
1 5 53
1 5 54
1 5 55
1 6 22
1 7 24
1 7 25
1 7 26


In [119]:
# 繪圖用
schedule_data = []
for i in I:
    for t in T:
        for k in K_i[i]:
            if y_ikt[i, k, t].x >0:
                #print(i,k,t)
                schedule_data.append([i, k, t])
              

In [120]:
# idle time
idle_time = 0
for i in I:
    #print(i,e_i[i].x,s_i[i].x,R_i[i])
    # print(i,e_i[i].x-s_i[i].x-R_i[i])
    idle_time += e_i[i].x-s_i[i].x-R_i[i]
print(idle_time)


12.0


In [121]:
# 超時項目
delay_time = 0
for i in I:
    for k in range(1,8):
        if u_ik[i, k].x >0:
            delay_time += u_ik[i, k].x
            print(i, k)
print(delay_time)


0


In [122]:
D_ik = pd.DataFrame(D_ik,
             columns = ['Register','MR','US','CT','Meal','Interpretation','Consultation'],
             index=list(I))

In [123]:
schedule_data

[[1, 1, 15],
 [1, 3, 17],
 [1, 4, 18],
 [1, 2, 20],
 [1, 5, 22],
 [1, 6, 22],
 [1, 7, 24],
 [2, 1, 1],
 [2, 2, 3],
 [2, 3, 5],
 [2, 4, 7],
 [2, 6, 8],
 [2, 5, 9],
 [2, 7, 11],
 [3, 1, 11],
 [3, 2, 13],
 [3, 3, 19],
 [3, 4, 23],
 [3, 6, 24],
 [3, 5, 26],
 [3, 7, 28],
 [4, 1, 2],
 [4, 2, 4],
 [4, 3, 10],
 [4, 6, 14],
 [4, 5, 16],
 [4, 7, 18],
 [5, 1, 4],
 [5, 2, 6],
 [5, 3, 12],
 [5, 4, 15],
 [5, 6, 15],
 [5, 7, 17],
 [6, 1, 16],
 [6, 2, 18],
 [6, 3, 24],
 [6, 6, 27],
 [6, 4, 28],
 [6, 7, 30],
 [7, 1, 21],
 [7, 2, 23],
 [7, 3, 29],
 [7, 6, 32],
 [7, 7, 34],
 [8, 1, 21],
 [8, 3, 23],
 [8, 2, 26],
 [8, 6, 32],
 [8, 7, 35],
 [9, 1, 9],
 [9, 2, 11],
 [9, 3, 17],
 [9, 6, 20],
 [9, 4, 21],
 [9, 7, 23],
 [10, 1, 25],
 [10, 3, 27],
 [10, 2, 30],
 [10, 6, 36],
 [10, 7, 38]]

In [124]:
D_ik.values.tolist()

[[2, 2, 1, 2, 2, 1, 3],
 [2, 2, 2, 2, 2, 1, 3],
 [2, 6, 4, 2, 2, 4, 3],
 [2, 6, 4, 0, 2, 4, 3],
 [2, 6, 3, 2, 0, 2, 3],
 [2, 6, 3, 2, 0, 3, 3],
 [2, 6, 3, 0, 0, 2, 3],
 [2, 6, 3, 0, 0, 3, 3],
 [2, 6, 3, 2, 0, 3, 3],
 [2, 6, 3, 0, 0, 2, 3]]

In [125]:
print("idle:",idle_time)
print("delay:",delay_time)
print("Obj:",model.ObjVal)

idle: 12.0
delay: 0
Obj: 12.0
